## Домашнее задание к лекции "Понятие класса"

### Задание 1

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса 
с помощью сервиса www.cbr-xml-daily.ru...ly_json.js


In [ ]:
import requests

def max_course():
    """
    Функция определяет на ресурсе: www.cbr-xml-daily.ru...ly_json.js
    max курс валюты и выводит название этой валюты
    """
    max_course = -1
    name_val = ''

    full_data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
    for line in full_data['Valute']:
        current_val = full_data['Valute'][line]['Value']
        if current_val > max_course:
            max_course = current_val
            name_val = full_data['Valute'][line]['Name']
     
    print(f'Валюта с максимальным значением курса: {name_val}')

max_course()

### Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. 
Считайте, self.diff будет принимать значение True только при возврате значения курса.
При отображении всей информации о валюте он не используется.


In [ ]:
class Rate:
    def __init__(self, format_='value', diff = False):
        self.format_ = format_
        if self.format_ == 'value':
            self.diff = diff
        else:
            # Cбрасываем self.diff в False при self.format_ != 'value'.
            # То есть при получении всей инфы, этот параметр не учитываем
            self.diff = False
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        'full' / 'value'   / 'prev'
        полная / тек. курс / пред. курс
        
        1 вариант:
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        2. вариант
        Rate('value').make_format('EUR')
        79.4966
        
        3. вариант
        Rate('prev').make_format('EUR')
        = предыд. значение курса
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format_ == 'full':
                return response[currency]
            
            if self.format_ == 'value':
                return response[currency]['Value']
            
            if  self.format_ == 'prev':
                return response[currency]['Previous']
        
        return 'Error'
    
    def eur(self):
        """
        Возвращает курс евро на сегодня в формате self.format_ 
        Если self.diff  = True возвращает предыдущее значение 
        """
        if self.diff:
            # Возвращаем округленную (разницу текущего курса с предыдущим)
            return round(Rate('value').make_format('EUR') - Rate('prev').make_format('EUR'),4)
        else:
            return self.make_format('EUR')

    
    def usd(self):
        """
        Возвращает курс доллара на сегодня в формате self.format_ 
        Если self.diff  = True возвращает предыдущее значение
        """
        if self.diff:
            # Возвращаем округленную (разницу текущего курса с предыдущим)
            return round(Rate('value').make_format('USD') - Rate('prev').make_format('USD'),4)

        else:
            return self.make_format('USD')

    def AZN(self):
        """
        Возвращает курс азербайджанского маната на сегодня в формате self.format_ 
        Если self.diff  = True возвращает предыдущее значение
        """
        if self.diff:
            # Возвращаем округленную (разницу текущего курса с предыдущим)
            return round(Rate('value').make_format('AZN') - Rate('prev').make_format('AZN'),4)
        else:
            return self.make_format('AZN')       

In [ ]:
a = Rate('full')
a.AZN()

In [ ]:
a = Rate('value')
a.AZN()

In [ ]:
a = Rate('full', True)
a.AZN()

In [ ]:
a = Rate('value', True)
a.AZN()

### Задание 3

Напишите класс Designer, который учитывает количество международных премий.
Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).
Класс Designer пишется по аналогии с классом Developer из материалов занятия.

Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [56]:
# Не совсем согласен с примером в лекции по разработчикам.
# В примере получается, что при получении от 1 до 4 баллов включительно у сотрудника уже есть 1 грейд
# Но это же не правильно. Пока сотрудник не получил 5 балов, Греды должны быть = 0
# Или это не так?


class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority        
        # Исправил, смотри коммент выше
        # self.grade = 1
        self.grade = 0      
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(f'Сотр-к: {self.name}; Баллы: {self.seniority}; Грейды: {self.grade}')
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [57]:
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [58]:
alex = Developer('Александр', 0)
for i in range(20):
    alex.check_if_it_is_time_for_upgrade()

Сотр-к: Александр; Баллы: 1; Грейды: 0
Сотр-к: Александр; Баллы: 2; Грейды: 0
Сотр-к: Александр; Баллы: 3; Грейды: 0
Сотр-к: Александр; Баллы: 4; Грейды: 0
Сотр-к: Александр; Баллы: 5; Грейды: 1
Сотр-к: Александр; Баллы: 6; Грейды: 1
Сотр-к: Александр; Баллы: 7; Грейды: 1
Сотр-к: Александр; Баллы: 8; Грейды: 1
Сотр-к: Александр; Баллы: 9; Грейды: 1
Сотр-к: Александр; Баллы: 10; Грейды: 2
Сотр-к: Александр; Баллы: 11; Грейды: 2
Сотр-к: Александр; Баллы: 12; Грейды: 2
Сотр-к: Александр; Баллы: 13; Грейды: 2
Сотр-к: Александр; Баллы: 14; Грейды: 2
Сотр-к: Александр; Баллы: 15; Грейды: 3
Сотр-к: Александр; Баллы: 16; Грейды: 3
Сотр-к: Александр; Баллы: 17; Грейды: 3
Сотр-к: Александр; Баллы: 18; Грейды: 3
Сотр-к: Александр; Баллы: 19; Грейды: 3
Сотр-к: Александр; Баллы: 20; Грейды: 4


In [59]:
class Designer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 2
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority += 2
        
        # условие повышения сотрудника из презентации
        # if self.seniority % 7 == 0:
        if (self.seniority - self.grade * 7) >= 7:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [60]:
# 4 - так как уже есть две премии, то заработанно 4 балла, поэтому передаю 4
Any = Designer('Any', 4)
# Две премии есть по умолчанию, поэтому интервал с тетьей премии...
for i in range(3,15):
    print(f'Получение международной премии № {i}.')
    Any.check_if_it_is_time_for_upgrade()

Получение международной премии № 3.
Сотр-к: Any; Баллы: 6; Грейды: 0
Получение международной премии № 4.
Сотр-к: Any; Баллы: 8; Грейды: 1
Получение международной премии № 5.
Сотр-к: Any; Баллы: 10; Грейды: 1
Получение международной премии № 6.
Сотр-к: Any; Баллы: 12; Грейды: 1
Получение международной премии № 7.
Сотр-к: Any; Баллы: 14; Грейды: 2
Получение международной премии № 8.
Сотр-к: Any; Баллы: 16; Грейды: 2
Получение международной премии № 9.
Сотр-к: Any; Баллы: 18; Грейды: 2
Получение международной премии № 10.
Сотр-к: Any; Баллы: 20; Грейды: 2
Получение международной премии № 11.
Сотр-к: Any; Баллы: 22; Грейды: 3
Получение международной премии № 12.
Сотр-к: Any; Баллы: 24; Грейды: 3
Получение международной премии № 13.
Сотр-к: Any; Баллы: 26; Грейды: 3
Получение международной премии № 14.
Сотр-к: Any; Баллы: 28; Грейды: 4
